# API Call for Flights Infos

Our task is to use Python to:
* get infos about flights through an API call to [Rapid API](https://rapidapi.com/hub/)
* extract from the json file the desired infos

In [2]:
# import libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import sqlalchemy
import pymysql
import requests
import config
from datetime import datetime,timedelta

In [3]:
# import df airports from SQL
host = config.host
schema = config.schema
user = config.user
password = config.password
port = config.port
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
airports_df=pd.read_sql("airports",con=con)

airports_df

,city_id,iata_code,airport_name
0,2,ARN,Stockholm Arlanda Airport
1,1,BER,Berlin Brandenburg Airport
2,2,BMA,Stockholm Bromma Airport


### API-call function

In [4]:
# introduce a function for the API call

def flights(airports_df):
    
    # parameters for the call (see url below in the for loop)
    now = datetime.now().date()
    tomorrow = (now + timedelta(days=1))
    start="08:00"
    end="20:00"

    # introduce an auxiliary list of dataframes
    df_list = []

    # performing the API call
    for i in range(len(airports_df)):
        
        # Prepare the request
        iata = airports_df.iloc[i,1]
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{tomorrow}T{start}/{tomorrow}T{end}"
        querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
        headers = {
            "content-type":config.content_type,
            "X-RapidAPI-Key": config.X_RapidAPI_Key,
            "X-RapidAPI-Host": config.X_RapidAPI_Host
        }
        response = requests.get(url, headers=headers, params=querystring)

        # convert the json file into a dataframe
        df_list.append(pd.json_normalize(response.json()["arrivals"]))

        # insert the IATA column
        df_list[i]['iata_code'] = iata

    #concatenate and keep only the relevant columns
    flight_info = pd.concat(df_list).reset_index()[['iata_code','number','status','departure.airport.name','arrival.scheduledTimeLocal','arrival.actualTimeLocal','arrival.scheduledTimeUtc','arrival.actualTimeUtc','arrival.terminal']]


    #convert arrival.actualTimeLocal and arrival.actualTimeUtc into date type
    flight_info['arrival.scheduledTimeLocal'] = pd.to_datetime(flight_info['arrival.scheduledTimeLocal'])
    flight_info['arrival.actualTimeLocal'] = pd.to_datetime(flight_info['arrival.actualTimeLocal'])
    flight_info['arrival.scheduledTimeUtc'] = pd.to_datetime(flight_info['arrival.scheduledTimeUtc'])
    flight_info['arrival.actualTimeUtc'] = pd.to_datetime(flight_info['arrival.actualTimeUtc'])
    
    #rename columns
    flight_info.columns = ['iata_code','flight_number','status','departure_city','scheduled_arrival_date_local','actual_arrival_date_local','scheduled_arrival_date_utc','actual_arrival_date_utc','terminal']
    
    
    return flight_info


flights_df = flights(airports_df)
flights_df

,iata_code,flight_number,status,departure_city,scheduled_arrival_date_local,actual_arrival_date_local,scheduled_arrival_date_utc,actual_arrival_date_utc,terminal
0,ARN,D8 3194,Expected,Copenhagen,2023-09-21 08:10:00+02:00,2023-09-21 08:10:00+02:00,2023-09-21 06:10:00+00:00,2023-09-21 06:10:00+00:00,5
1,ARN,D8 4030,Expected,Luleå,2023-09-21 08:10:00+02:00,2023-09-21 08:10:00+02:00,2023-09-21 06:10:00+00:00,2023-09-21 06:10:00+00:00,5
2,ARN,DL 204,Expected,New York,2023-09-21 08:00:00+02:00,2023-09-21 08:00:00+02:00,2023-09-21 06:00:00+00:00,2023-09-21 06:00:00+00:00,5
3,ARN,DS 1581,Expected,Geneva,2023-09-21 08:55:00+02:00,2023-09-21 08:55:00+02:00,2023-09-21 06:55:00+00:00,2023-09-21 06:55:00+00:00,2
4,ARN,DY 804,Expected,Oslo,2023-09-21 08:40:00+02:00,2023-09-21 08:40:00+02:00,2023-09-21 06:40:00+00:00,2023-09-21 06:40:00+00:00,5
...,...,...,...,...,...,...,...,...,...
545,BMA,TF 321,Expected,Ängelholm,2023-09-21 19:15:00+02:00,2023-09-21 19:15:00+02:00,2023-09-21 17:15:00+00:00,2023-09-21 17:15:00+00:00,1
546,BMA,TF 35,Expected,Goteborg,2023-09-21 19:30:00+02:00,2023-09-21 19:30:00+02:00,2023-09-21 17:30:00+00:00,2023-09-21 17:30:00+00:00,1
547,BMA,TF 3921,Expected,Helsinki,2023-09-21 19:05:00+02:00,2023-09-21 19:05:00+02:00,2023-09-21 17:05:00+00:00,2023-09-21 17:05:00+00:00,1
548,BMA,TF 427,Expected,Visby,2023-09-21 19:20:00+02:00,2023-09-21 19:20:00+02:00,2023-09-21 17:20:00+00:00,2023-09-21 17:20:00+00:00,1


### Sending results to SQL

In [3]:
flights_df.to_sql('flights',con=con,if_exists='append',index=False)

550